In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/flower-recognition-he/he_challenge_data/data"))

### Need pretrainedmodels to load the pretrained Cedene models into fastai.
### https://github.com/Cadene/pretrained-models.pytorch

In [ ]:
!pip install pretrainedmodels

In [ ]:
from torchvision.models import *
import pretrainedmodels as pm
!pip install pretrainedmodels
from fastai import *
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import fastai

from utils import *
import sys
import torch
fastai.__version__

In [ ]:
pm.__dict__

In [ ]:
def se_resnext50_32x4d(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pm.xception(pretrained=pretrained)
    return nn.Sequential(*list(model.children()))

In [ ]:
lis = os.listdir('../input/flower-recognition-he/he_challenge_data/data/train')

In [ ]:
sub = pd.read_csv('../input/flower-recognition-he/he_challenge_data/data/sample_submission.csv')

In [ ]:
sub.shape

In [ ]:
bs=8

In [ ]:
## test filenames to be used to create final submission.
filenames = os.listdir('../input/flower-recognition-he/he_challenge_data/data/test')

In [ ]:
df= pd.read_csv('../input/flower-recognition-he/he_challenge_data/data/train.csv')

In [ ]:
df.head()

In [ ]:
df.image_id=df.image_id.apply(lambda x:'train'+'/'+str(x)+'.jpg')

In [ ]:
im_count = df.category.value_counts()

In [ ]:
classes=df.category.unique()

In [ ]:
k=[]
for i in range(len(classes)):
    q=df[df.category.values==classes[i]].image_id.values[0:20]
    for j in range(len(q)):
        k.append(q[j])
k=np.array(k)

In [ ]:
k[i]

In [ ]:
path="../input/flower-recognition-he/he_challenge_data/data"

In [ ]:
SZ = 224
BS = 64
NUM_WORKERS = 12
SEED=0

In [ ]:
# CenterCrop(32)
## These Transformation applied based upon my previous competition Experience.
## if you want to try other transformation check this link
## https://docs.fast.ai/vision.transform.html
tfms = get_transforms(max_zoom=1.9,max_rotate=80.0,
                      max_warp=0.2, p_affine=0.75,do_flip=True,xtra_tfms=[cutout()])
tfms1 = tfms
data = (ImageList.from_csv(path, folder = 'train', suffix='.jpg',csv_name='train.csv')
        .split_by_rand_pct(.2,seed=42)              
        .label_from_df()            
        .add_test_folder(test_folder = 'test')              
        .transform(tfms, size=SZ)
        .databunch(num_workers=12,bs=BS))

In [ ]:
## to see the images in train with there labels
data.show_batch(rows=3, figsize=(8,10))

In [ ]:
from torch import nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduction='elementwise_mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction is None:
            return F_loss
        else:
            return torch.mean(F_loss)

In [ ]:
## load the pretrained imagenet model
## you can try other models from this link
## https://docs.fast.ai/vision.models.html
learn = cnn_learner(data, models.resnet50, ps=0,metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn.loss_fn=FocalLoss()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=3e-3
learn.fit_one_cycle(5,slice(lr,lr/10),wd=1e-4)

In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
learn1 = cnn_learner(data, models.resnet50, ps=0,metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn1.loss_fn=FocalLoss()
learn1.lr_find()
learn1.recorder.plot()

In [ ]:
learn1.fit_one_cycle(3,slice(3e-3),wd=1e-2)

In [ ]:
labelled_preds = []
pred13 = preds
for pred in pred13:
    labelled_preds.append(int(np.argmax(pred))+1)

submission1 = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission1.to_csv('submission1.csv',index=False)

In [ ]:
models.__dict__

In [ ]:
learn5 = cnn_learner(data,models.resnet34, ps=0.1,metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn5.fit_one_cycle(1)
learn5.unfreeze()
learn5.lr_find()
learn5.fit_one_cycle(5, max_lr=slice(1e-6,1e-3))

In [ ]:
preds5,_ = learn5.TTA(ds_type=DatasetType.Test)

In [ ]:
preds5

In [ ]:
import pickle

filehandler = open("preds5.pkl","wb")
pickle.dump(preds5,filehandler)
filehandler.close()

In [ ]:
labelled_preds = []
pred13 = preds5
for pred in pred13:
    labelled_preds.append(int(np.argmax(pred))+1)

submission2 = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission2.to_csv('submission2.csv',index=False)

In [ ]:
learn1 = cnn_learner(data1, models.resnet152,ps=0, metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn1.fit_one_cycle(1)
learn1.unfreeze()
learn1.lr_find()
learn1.fit_one_cycle(11, max_lr=slice(1e-5,1e-3))

In [ ]:
preds1,_ = learn1.TTA(ds_type=DatasetType.Test)

In [ ]:
filehandler = open("preds1.pkl","wb")
pickle.dump(preds1,filehandler)
filehandler.close()

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download pkl file", filename = "preds1.pkl"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename ='preds1.pkl')

In [ ]:
learn2 = cnn_learner(data1, models.densenet121,ps=0, metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn2.fit_one_cycle(1)
learn2.unfreeze()
learn2.lr_find()
learn2.fit_one_cycle(12, max_lr=slice(1e-6,1e-3))

In [ ]:
learn4 = cnn_learner(data1, models.vgg16_bn,ps=0, metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn4.fit_one_cycle(1)
learn4.unfreeze()
learn4.lr_find()
learn4.fit_one_cycle(8, max_lr=slice(1e-6,1e-3))
preds4,_ = learn4.TTA(ds_type=DatasetType.Test)

In [ ]:
learn6 = cnn_learner(data1,models.resnet50 , ps=0,metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn6.fit_one_cycle(1)
learn6.unfreeze()
learn6.lr_find()
learn6.fit_one_cycle(13, max_lr=slice(1e-6,1e-3))

In [ ]:
preds6,_ = learn6.TTA(ds_type=DatasetType.Test)

In [ ]:
filehandler = open("preds6.pkl","wb")
pickle.dump(preds6,filehandler)
filehandler.close()
from IPython.display import HTML
def create_download_link(title = "Download pkl file", filename = "preds6.pkl"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename ='preds6.pkl')

In [ ]:
labelled_preds = []
pred13 = preds6
for pred in pred13:
    labelled_preds.append(int(np.argmax(pred))+1)

submission4 = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission4.to_csv('submission4.csv',index=False)

In [ ]:
learn3 = cnn_learner(data, models.resnet101,ps=0, metrics=[error_rate, accuracy], model_dir="/tmp/model/")
## training with one cycle which used cyclic learning rate and learning rate annhelling
learn3.fit_one_cycle(1)
learn3.unfreeze()
learn3.lr_find()
learn3.fit_one_cycle(13, max_lr=slice(1e-5,1e-3))
preds3,_ = learn3.TTA(ds_type=DatasetType.Test)

In [ ]:
preds3,_ = learn3.TTA(ds_type=DatasetType.Test)

In [ ]:
filehandler = open("preds3.pkl","wb")
pickle.dump(preds3,filehandler)
filehandler.close()
from IPython.display import HTML
def create_download_link(title = "Download pkl file", filename = "preds3.pkl"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename ='preds3.pkl')

In [ ]:
labelled_preds = []
pred11 = preds3
for pred in pred11:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission.to_csv('submission5.csv',index=False)

In [ ]:
a="../input/stacke/preds3.pkl"
b="../input/stacke/preds6.pkl"
c="../input/stacke/preds5.pkl"

In [ ]:
def unpick(file):
    infile = open(file,'rb')
    new_dict = pickle.load(infile)
    infile.close()
    return new_dict

In [ ]:
a=unpick(a)
b=unpick(b)
c=unpick(c)

In [ ]:
labelled_preds = []
predict = (a+b+c)/3
for pred in predict:
    labelled_preds.append(int(np.argmax(pred))+1)

submission3 = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission3.to_csv('submission.csv',index=False)

In [ ]:
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])
submission1['image_id'] = submission1['image_id'].apply(lambda x:x.split('.')[0])
submission2['image_id'] = submission2['image_id'].apply(lambda x:x.split('.')[0])
submission3['image_id'] = submission3['image_id'].apply(lambda x:x.split('.')[0])
submission4['image_id'] = submission4['image_id'].apply(lambda x:x.split('.')[0])

In [ ]:
## To download the submission file without Commiting the kernel.

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "subm.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission2)

In [ ]:
## This is the Fastai Baseline for the Flower Recognition Competition.If you like the work then plz Upvote the Kernel.
## https://www.hackerearth.com/challenges/competitive/garden-nerd

In [ ]:
#keras